## The Power OF Function Calling using OpenAI API

In [1]:
pip install openai


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install langchain

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
from openai import AzureOpenAI

In [4]:
pip install azure-identity

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [6]:
pip install pydantic

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install instructor==0.3.1

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from azure.identity import ManagedIdentityCredential
from azure.keyvault.secrets import SecretClient
 

In [ ]:
VAULT_URL="https://bswhivekeyvault.vault.azure.net"


In [ ]:

from azure.identity import DefaultAzureCredential


In [ ]:


# system-assigned managed identity
cred = DefaultAzureCredential()
 
#VAULT_URL = os.environ["AZURE_KEYVAULT_RESOURCEENDPOINT"]
secret_client = SecretClient(vault_url=VAULT_URL, credential=cred)
 
AZURE_OPENAI_SECRET = secret_client.get_secret("llm-dev-02")
assert AZURE_OPENAI_SECRET, "ERROR: Azure OpenAI Key is missing"
 
AZURE_OPENAI_ENDPOINT = "https://llm-dev-02.openai.azure.com/"
AZURE_OPENAI_API_VERSION = "2023-05-15"
CHAT_DEPLOYMENT_NAME =  "gpt4-turbo-askbsw"
EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada02-askbsw"
 
SEARCH_SERVICE_ENDPOINT = "https://cog-askbsw-dev.search.windows.net/"
SEARCH_SERVICE_API_SECRET = secret_client.get_secret("searchserviceapikey")
assert SEARCH_SERVICE_API_SECRET, "ERROR: Search Service Key is missing"
 
INDEX_NAME = "askms-embeddings-index-v2"
TEMPERATURE = 0.01
 
MESSAGE_BUFFER = 4
 
VECTOR_SEARCH_SCORE_THRESHOLD = 0.81

In [8]:
import os
import openai
#.api_key=openai_key

In [9]:
#from dotenv import load_dotenv
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
#load_dotenv


/Users/balasubramaniamdakshinamoorthi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [10]:

AZURE_OPENAI_API_KEY="8cf7f12ce06d4dd387dcdc3e8e4594fc"

In [11]:
AZURE_OPENAI_ENDPOINT="https://llm-dev-02.openai.azure.com"

In [12]:
from pydantic import Field
from typing import Optional
import instructor
import openai


class Validator(instructor.OpenAISchema):
    """
    Validate if an attribute is correct and if not,
    return a new value with an error message
    """

    is_valid: bool = Field(
        default=True,
        description="Whether the attribute is valid based on the requirements",
    )
    reason: Optional[str] = Field(
        default=None,
        description="The error message if the attribute is not valid, otherwise None",
    )
    fixed_value: Optional[str] = Field(
        default=None,
        description="If the attribute is not valid, suggest a new value for the attribute",
    )


def llm_validator(
    statement: str,
    allow_override: bool = False,
    engine: str = "gpt-4",
    temperature: float = 0,
):
    """
    Create a validator that uses the LLM to validate an attribute

    ## Usage

    ```python
    from instructor import llm_validator
    from pydantic import BaseModel, Field, field_validator

    class User(BaseModel):
        name: str = Annotated[str, llm_validator("The name must be a full name all lowercase")]
        age: int = Field(description="The age of the person")

    try:
        user = User(name="Jason Liu", age=20)
    except ValidationError as e:
        print(e)
    ```

    ```
    1 validation error for User
    name
      The name is valid but not all lowercase (type=value_error.llm_validator)
    ```

    Note that there, the error message is written by the LLM, and the error type is `value_error.llm_validator`.

    Parameters:
        statement (str): The statement to validate
        model (str): The LLM to use for validation (default: "gpt-3.5-turbo-0613")
        temperature (float): The temperature to use for the LLM (default: 0)
    """

    def llm(v):
        resp = openai.ChatCompletion.create(
            functions=[Validator.openai_schema],
            function_call={"name": Validator.openai_schema["name"]},
            messages=[
                {
                    "role": "system",
                    "content": "You are a world class validation model. Capable to determine if the following value is valid for the statement, if it is not, explain why and suggest a new value.",
                },
                {
                    "role": "user",
                    "content": f"Does `{v}` follow the rules: {statement}",
                },
            ],
            engine=engine,
            temperature=temperature,
        )  # type: ignore
        resp = Validator.from_response(resp)

        # If the response is  not valid, return the reason, this could be used in
        # the future to generate a better response, via reasking mechanism.
        assert resp.is_valid, resp.reason

        if allow_override and not resp.is_valid and resp.fixed_value is not None:
            # If the value is not valid, but we allow override, return the fixed value
            return resp.fixed_value
        return v

    return llm


In [ ]:
Azureopenaiclient= AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-07-01-preview",
    azure_deployment="mybswhealth"
    
)

In [ ]:
azureopenaipatchebswsclient = instructor.patch( AzureOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2023-07-01-preview",
    azure_deployment="mybswhealth"
    
))

In [ ]:
import enum
import json
from pydantic import BaseModel

In [ ]:
from typing import List
import enum
import json

# Define Enum class for multiple labels
class MultiLabels(str, enum.Enum):
    NEW_SYMPTOMS="Experiencing new symptoms"
    COMPLEX_SYMPTOMS="Known chronic or serious health conditions,Recent or ongoing medical treatment body systems"
    MIGRAINE_ELIGIBILITY="Headache,head pain, migraines, long term sufferer of headaches"
    BACK_AND_NECK_ELIGIBILITY="Experiencing back pain or neck pain or stiffness" 
    MEDICATION_REFILLS="Refills just want help booking an appointment,Have a referral"
    PRESCRIPTION_RENEWALS="prescription"
    ASSISTED_SCHEDULING="schedule"
    FIND_CARE="I need a care "
    SERVICES="I need labs, imaging, clinical testing"
    BILL_SUPPORT="my current bill"
    COST_ESTIMATION="I need to know how much cost"
    PRICE_SHOPPING="price"
    GENERAL_HEALTH_SYSTEM_NAVIGATION="hospital visiting hours"
    RED_FLAG="not any"
    NONE_OF_THE_ABOVE="none"


# Define the multi-class prediction model
class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """

    class_labels: List[MultiLabels]

In [ ]:
from openai import OpenAI
import instructor

# Apply the patch to the OpenAI client
# enables response_model keyword

def multi_classify(data: str) -> MultiClassPrediction:
    """Perform multi-label classification on the input text."""
    return azureopenaipatchebswsclient.chat.completions.create(
        model="gpt-4",
        response_model=MultiClassPrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following issue: {data}",
            },
        ],
    )  # type: ignore

In [ ]:
# Test multi-label classification
issue= "chronic ."
predictionissues = multi_classify(issue)
try: 
   assert MultiLabels.NEW_SYMPTOMS in predictionissues.class_labels
   assert MultiLabels.COMPLEX_SYMPTOMS in predictionissues.class_labels
   assert MultiLabels.BACK_AND_NECK_ELIGIBILITY in predictionissues.class_labels
   assert MultiLabels.MEDICATION_REFILLS in predictionissues.class_labels
   assert MultiLabels.PRESCRIPTION_RENEWALS in predictionissues.class_labels
   assert MultiLabels.ASSISTED_SCHEDULING in predictionissues.class_labels
   assert MultiLabels.FIND_CARE in predictionissues.class_labels
   assert MultiLabels.SERVICES in predictionissues.class_labels
   assert MultiLabels.BILL_SUPPORT in predictionissues.class_labels
   assert MultiLabels.COST_ESTIMATION in predictionissues.class_labels
   assert MultiLabels.PRICE_SHOPPING in predictionissues.class_labels
   assert MultiLabels.GENERAL_HEALTH_SYSTEM_NAVIGATION in predictionissues.class_labels
   assert MultiLabels.RED_FLAG in predictionissues.class_labels
   assert MultiLabels.NONE_OF_THE_ABOVE in predictionissues.class_labels
# the errror_message provided by the user gets printed 
except AssertionError as messages: 
    print(messages)

In [ ]:

print(predictionissues)

In [ ]:
response = Azureclient.chat.completions.create(
    model="gpt-35-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    #temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")


In [ ]:
# For all possible arguments see https://platform.openai.com/docs/api-reference/chat-completions/create
response = openAIclient.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
)

print(f"{response.choices[0].message.role}: {response.choices[0].message.content}")


In [ ]:
response = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Knock knock."},
        {"role": "assistant", "content": "Who's there?"},
        {"role": "user", "content": "Orange."},
    ],
    temperature=0,
    stream=True
)

for chunk in response:
    if len(chunk.choices) > 0:
        delta = chunk.choices[0].delta

        if delta.role:
            print(delta.role + ": ", end="", flush=True)
        if delta.content:
            print(delta.content, end="", flush=True)


In [ ]:
import json

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "<text violating the content policy>"}
]

try:
    completion = client.chat.completions.create(
        messages=messages,
        model=deployment,
    )
except openai.BadRequestError as e:
    err = json.loads(e.response.text)
    if err["error"]["code"] == "content_filter":
        print("Content filter triggered!")
        content_filter_result = err["error"]["innererror"]["content_filter_result"]
        for category, details in content_filter_result.items():
            print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")


In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the biggest city in Washington?"}
]

completion = client.chat.completions.create(
    messages=messages,
    model=deployment,
)
print(f"Answer: {completion.choices[0].message.content}")

# prompt content filter result in "model_extra" for azure
prompt_filter_result = completion.model_extra["prompt_filter_results"][0]["content_filter_results"]
print("\nPrompt content filter results:")
for category, details in prompt_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

# completion content filter result
print("\nCompletion content filter results:")
completion_filter_result = completion.choices[0].model_extra["content_filter_results"]
for category, details in completion_filter_result.items():
    print(f"{category}:\n filtered={details['filtered']}\n severity={details['severity']}")

In [ ]:
pip install instructor==0.3.1

In [ ]:
import enum
from pydantic import BaseModel


class Labels(str, enum.Enum):
    """Enumeration for single-label text classification."""

    SPAM = "spam"
    NOT_SPAM = "not_spam"


class SinglePrediction(BaseModel):
    """
    Class for a single class label prediction.
    """

    class_label: Labels

In [ ]:
from pydantic import Field
from typing import Optional
import instructor
import openai


class Validator(instructor.OpenAISchema):
    """
    Validate if an attribute is correct and if not,
    return a new value with an error message
    """

    is_valid: bool = Field(
        default=True,
        description="Whether the attribute is valid based on the requirements",
    )
    reason: Optional[str] = Field(
        default=None,
        description="The error message if the attribute is not valid, otherwise None",
    )
    fixed_value: Optional[str] = Field(
        default=None,
        description="If the attribute is not valid, suggest a new value for the attribute",
    )


def llm_validator(
    statement: str,
    allow_override: bool = False,
    engine: str = "gpt-35-turbo",
    temperature: float = 0,
):
    """
    Create a validator that uses the LLM to validate an attribute

    ## Usage

    ```python
    from instructor import llm_validator
    from pydantic import BaseModel, Field, field_validator

    class User(BaseModel):
        name: str = Annotated[str, llm_validator("The name must be a full name all lowercase")]
        age: int = Field(description="The age of the person")

    try:
        user = User(name="Jason Liu", age=20)
    except ValidationError as e:
        print(e)
    ```

    ```
    1 validation error for User
    name
      The name is valid but not all lowercase (type=value_error.llm_validator)
    ```

    Note that there, the error message is written by the LLM, and the error type is `value_error.llm_validator`.

    Parameters:
        statement (str): The statement to validate
        model (str): The LLM to use for validation (default: "gpt-3.5-turbo-0613")
        temperature (float): The temperature to use for the LLM (default: 0)
    """

    def llm(v):
        resp = openai.ChatCompletion.create(
            functions=[Validator.openai_schema],
            function_call={"name": Validator.openai_schema["name"]},
            messages=[
                {
                    "role": "system",
                    "content": "You are a world class validation model. Capable to determine if the following value is valid for the statement, if it is not, explain why and suggest a new value.",
                },
                {
                    "role": "user",
                    "content": f"Does `{v}` follow the rules: {statement}",
                },
            ],
            engine=engine,
            temperature=temperature,
        )  # type: ignore
        resp = Validator.from_response(resp)

        # If the response is  not valid, return the reason, this could be used in
        # the future to generate a better response, via reasking mechanism.
        assert resp.is_valid, resp.reason

        if allow_override and not resp.is_valid and resp.fixed_value is not None:
            # If the value is not valid, but we allow override, return the fixed value
            return resp.fixed_value
        return v

    return llm


In [ ]:
pip install pydantic

In [ ]:
pip install instructor==0.3.1

In [ ]:
import enum
from pydantic import BaseModel

In [ ]:
from typing import List
import enum

# Define Enum class for multiple labels
class MultiLabels(str, enum.Enum):
    TECH_ISSUE = "tech_issue"
    BILLING = "billing"
    GENERAL_QUERY = "general_query"


# Define the multi-class prediction model
class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """

    class_labels: List[MultiLabels]

In [ ]:
from pydantic import BaseModel, ValidationError, BeforeValidator
from typing_extensions import Annotated
from instructor import llm_validator



from typing import List
import enum

# Define Enum class for multiple labels
class MultiLabels(str, enum.Enum):
    TECH_ISSUE = "tech_issue"
    BILLING = "billing"
    GENERAL_QUERY = "general_query"


# Define the multi-class prediction model
class MultiClassPrediction(BaseModel):
    """
    Class for a multi-class label prediction.
    """

    class_labels: List[MultiLabels]

In [ ]:
def multi_classify(data: str) -> MultiClassPrediction:
    """Perform multi-label classification on the input text."""
    return azureopenaipatchclient .chat.completions.create(
        model="gpt-3.5-turbo-0613",
        response_model=MultiClassPrediction,
        messages=[
            {
                "role": "user",
                "content": f"Classify the following support ticket: {data}",
            },
        ],
    )  # type: ignore

In [ ]:
# Test multi-label classification
ticket = "My account is locked and I can't access my billing info."
prediction = multi_classify(ticket)
assert MultiLabels.TECH_ISSUE in prediction.class_labels
assert MultiLabels.BILLING in prediction.class_labels

In [ ]:

print(prediction)

In [ ]:
# Test multi-label classification
ticket = "My bill is came abnormally high this month."
prediction1 = multi_classify(ticket)
assert MultiLabels.TECH_ISSUE in prediction.class_labels
assert MultiLabels.BILLING in prediction.class_labels

In [ ]:
print(prediction1)

In [ ]:
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
import requests
def get_current_weather(location):
    """Get the current weather in a given location"""

    url = "https://ai-weather-by-meteosource.p.rapidapi.com/find_places"

    querystring = {"text":location}

    headers = {
      "X-RapidAPI-Key": "XXX",
      "X-RapidAPI-Host": "XXX"
    }

    response = requests.get(url, headers=headers, params=querystring)

    print(response.json())
  
    return response.json()


In [ ]:
response=get_current_weather('Bangalore')

In [ ]:
response

In [ ]:
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    
                },
                "required": ["location"],
            },
        }
    ]

In [ ]:
functions

In [ ]:
print(completion.choices[0].message)

In [ ]:
response=completion.choices[0].message

In [ ]:
response

In [ ]:
function_name=response['function_call']['name']
print(function_name)


In [ ]:
import json
location=eval(response['function_call']['arguments'])['location']
print(location)

In [ ]:
# Step 4: send the info on the function call and function response to GPT
messages.append(response)  # extend conversation with assistant's reply
messages.append(
    {
        "role": "function",
        "name": function_name,
        "content": location,
    }
)

In [ ]:
messages

In [ ]:
# extend conversation with function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)  # get a new response from GPT where it can see the function response



In [ ]:
print(second_response.choices[0].message)

In [ ]:
second_response